In [15]:
import pandas as pd
import numpy as np
import folium
import math


from statistics import mean 
from IPython.display import Markdown
from opencage.geocoder import OpenCageGeocode


def special_print(title, content):
    display(Markdown("### %s:" % title))
    print(content)
    return None
    
    
class getBiodiversity():

    def __init__(self, url):
        self.url = url
        try:
            self.df_data = pd.read_csv(url, sep=';', header=0, encoding='utf-8')
        except Exception as e:
            self.df_data = pd.DataFrame()
            print("Aborting... couldn't read this file: %s" % url)
            print (e.args)
        self.data_info = "File shape: %d rows x %d columns"% (self.df_data.shape[0], self.df_data.shape[1])
        return None
    
    def getColumns(self):
        self.df_columns = list(self.df_data.columns)
        return None
    
    def checkEmpty(self):
        self.getColumns()
        self.df_dataNAN = pd.DataFrame(np.where((self.df_data == '') | (self.df_data == 'Sem Informações'), 1, 0))
        self.df_dataNAN.columns = self.df_columns
        self.df_data_missing = 100*self.df_dataNAN.mean()
        return None

    def getLastFilled(self, columns):
        filled_columns = [column for column in columns if (column != "Sem Informações")]
        return 'NA' if len(filled_columns) == 0 else TAXONOMY_COLUMNS[len(filled_columns)-1]
    
    def addTaxonomicLevel(self, col_name):
        self.df_data[col_name] = self.df_data[TAXONOMY_COLUMNS].apply(lambda x: self.getLastFilled(x), axis=1)
        self.df_taxonomy_info =  self.df_data[col_name].value_counts()
        return None

    def extractTaxonomy(self, columns):
        self.df_taxonomy = self.df_data[columns].copy()
        return None
    
    def getTaxonomy(self, col_name='taxonomic_level'):
        self.addTaxonomicLevel(col_name)
        self.extractTaxonomy(TAXONOMY_COLUMNS+[col_name])
        return None
    
    def filterFields(self, columns, values):
        filter = np.logical_and.reduce([self.df_data[columns[i]].isin(values[i]) for i in range(len(columns))])
        self.df_filtered = self.df_data[filter].copy()
        self.filtered_info = "File shape: %d rows x %d columns"% (self.df_filtered.shape[0], self.df_filtered.shape[1])
        return None
    
    def parseFloat(self, info):
        value = float(info)
        try:
            value = float(info)
        except:
            value = 0.0
        return value
    
    def checkGeoInfo(self, components, reported):
        aux = []
        unmatched = 0
        for elem in ["country", "state", "state_code", "city"]:
            try:
                value = components[elem]
            except:
                value = "NA"
            aux.append(value)
        unmatched += 1 if reported[0] != aux[0] else 0
        unmatched += 1 if not reported[1] in [aux[1], aux[2]] else 0
        unmatched += 1 if reported[2] != aux[3] else 0
        return unmatched
    
    def reverseGeocode(self, latlon):
        geo = geocoder.reverse_geocode(latlon[0], latlon[1], no_annotations = '1', pretty = '1', language='pt')
        comp = geo[0]['components']
        info = self.checkGeoInfo(comp, [latlon[2], latlon[3], latlon[4]])
        return pd.Series((geo[0]['formatted'], info))
    
    def setMapZoom(self, coords):
        try:
            rangelat = math.sqrt(170 / (max(coords[0][:])-min(coords[0][:])))
            rangelon = math.sqrt(360 / (max(coords[1][:])-min(coords[1][:])))
            zoom = int(min(rangelat, rangelon)) + 1
        except:
            zoom = 1
        return zoom
    
    def printMap(self):
        coords = self.df_location_sample[["AdjustedLatitude", "AdjustedLongitude", "ReversedAddress", "Confidence"]].T.values.tolist()
        COLORS = ['green', 'lightgreen', 'orange', 'red']
        center = [mean(coords[0][:]), mean(coords[1][:])]
        zoom = self.setMapZoom(coords[0:2][:])
        my_map = folium.Map(location=center, zoom_start=zoom, tiles="OpenStreetMap")
        for i in range(len(self.df_location_sample)):
            folium.Marker(location=[coords[0][i], coords[1][i]], popup=coords[2][i], 
                          icon=folium.Icon(color=COLORS[coords[3][i]], icon='map-marker')).add_to(my_map) 
        self.observations_map = my_map
        return None
        
    def checkCoordinates(self, size):
        self.df_filtered["AdjustedLatitude"] = self.df_data["Latitude"].apply(lambda x: self.parseFloat(x))
        self.df_filtered["AdjustedLongitude"] = self.df_data["Longitude"].apply(lambda x: self.parseFloat(x))
        if len(self.df_filtered) < size:
            print("Not enough data to show. Please check your filter opetions")
            self.df_location_sample = pd.DataFrame()
            self.observations_map = None
            return None
        self.df_location_sample = self.df_filtered.sample(n=size).copy()
        self.df_location_sample[['ReversedAddress','Confidence']] = self.df_location_sample[['AdjustedLatitude','AdjustedLongitude']+LOCATION_COORDINATES].apply(self.reverseGeocode, axis=1)
        self.printMap()
        return None



####################################################################################
#
# Initializing all data

#url = "portalbio_export_16-10-2019-14-39-54.csv"
#url = "PortalBio 00002.csv"
url = "PortalBio 00043.csv"
#url = "PortalBio 00555.csv"
#url = "PortalBio 03912.csv"
#url = "PortalBio 58411.csv"
TAXONOMY_COLUMNS = ['Filo', 'Classe', 'Ordem', 'Familia', 'Genero', 'Especie']
LOCATION_COORDINATES = ['Pais', 'Estado/Provincia', 'Municipio', 'Latitude', 'Longitude']
LOCATION_SAMPLING = 5 # number of locations to check

FILTER_FIELDS = ['Municipio','Filo']
FILTER_VALUES = [['Nova Friburgo','Niquelândia','Vitoria','Primavera do Leste'],['Mollusca','Magnoliophyta', 'Arthropoda']]

key = '09aadb1b1d8840acacfa0fcece0acb13'
geocoder = OpenCageGeocode(key)

biodiversity = getBiodiversity(url)
biodiversity.checkEmpty()
biodiversity.getTaxonomy(col_name='Nível Taxonômico')
biodiversity.filterFields(FILTER_FIELDS, FILTER_VALUES)
biodiversity.checkCoordinates(LOCATION_SAMPLING)

In [16]:
####################################################################################
#
# Show sample of each output - raw data load

special_print("File URL", biodiversity.url)
special_print("Raw file info", biodiversity.data_info)
special_print("Raw file sample", biodiversity.df_data.head(1).T)
special_print("Dataframe columns", biodiversity.df_columns)

### File URL:

PortalBio 00043.csv


### Raw file info:

File shape: 43 rows x 34 columns


### Raw file sample:

                                                                                     0
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                Sistema de Autorização e Informação em Biodive...
Sigla da base de dados                                                    SISBIO-DIBIO
Responsavel pelo registro                          Rose Gomes Monnerat Solon De Pontes
Numero do registro no portal                                                    812136
Numero do registro na base de dados            Nº Da Autorização/Licença Sisbio: 27130
Data do registro                                                            01/11/2015
Data do evento                                                 02/11/2015 a 23/11/2015
Data de Carencia                                                            06/01/2016
Nome cientifico                            

### Dataframe columns:

['Nome da instituicao', 'Sigla da instituicao', 'Nome da base de dados', 'Sigla da base de dados', 'Responsavel pelo registro', 'Numero do registro no portal', 'Numero do registro na base de dados', 'Data do registro', 'Data do evento', 'Data de Carencia', 'Nome cientifico', 'Nome comum', 'Nome cientifico na base de dados', 'Nivel taxonomico', 'Numero de individuos', 'Reino', 'Filo', 'Classe', 'Ordem', 'Familia', 'Genero', 'Especie', 'Estado de conservacao', 'Categoria de Ameaca', 'Localidade', 'Pais', 'Estado/Provincia', 'Municipio', 'Status de Sensibilidade', 'Latitude', 'Longitude', 'Outras informacoes da localidade', 'Jurisdicao', 'Destino do Material']


In [17]:
####################################################################################
#
# Show sample of each output - data missing analysis

special_print("Data missing sample (1 = missing)", biodiversity.df_dataNAN.head(5).T)
special_print("Data missing statistics (%)", biodiversity.df_data_missing)

### Data missing sample (1 = missing):

                                     0  1  2  3  4
Nome da instituicao                  1  1  1  1  1
Sigla da instituicao                 0  0  0  0  0
Nome da base de dados                0  0  0  0  0
Sigla da base de dados               0  0  0  0  0
Responsavel pelo registro            0  0  0  0  0
Numero do registro no portal         0  0  0  0  0
Numero do registro na base de dados  0  0  0  0  0
Data do registro                     0  0  0  0  0
Data do evento                       0  0  0  0  0
Data de Carencia                     0  0  0  0  0
Nome cientifico                      0  0  0  0  0
Nome comum                           1  1  1  1  1
Nome cientifico na base de dados     0  0  0  0  0
Nivel taxonomico                     0  0  0  0  0
Numero de individuos                 0  0  0  0  0
Reino                                1  1  1  1  1
Filo                                 0  0  0  0  0
Classe                               0  0  0  0  0
Ordem                          

### Data missing statistics (%):

Nome da instituicao                    100.00000
Sigla da instituicao                     0.00000
Nome da base de dados                    0.00000
Sigla da base de dados                   0.00000
Responsavel pelo registro                0.00000
Numero do registro no portal             0.00000
Numero do registro na base de dados      0.00000
Data do registro                         0.00000
Data do evento                           0.00000
Data de Carencia                         0.00000
Nome cientifico                          0.00000
Nome comum                             100.00000
Nome cientifico na base de dados         0.00000
Nivel taxonomico                         0.00000
Numero de individuos                     0.00000
Reino                                  100.00000
Filo                                     0.00000
Classe                                   0.00000
Ordem                                    0.00000
Familia                                  0.00000
Genero              

In [18]:
####################################################################################
#
# Show sample of each output - show taxonomic info

special_print("Raw data sample after taxonomic level inclusion", biodiversity.df_data.head(1).T)
special_print("Taxonomic info", biodiversity.df_taxonomy_info)
special_print("Taxonomy sample", biodiversity.df_taxonomy.head(3).T)

### Raw data sample after taxonomic level inclusion:

                                                                                     0
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                Sistema de Autorização e Informação em Biodive...
Sigla da base de dados                                                    SISBIO-DIBIO
Responsavel pelo registro                          Rose Gomes Monnerat Solon De Pontes
Numero do registro no portal                                                    812136
Numero do registro na base de dados            Nº Da Autorização/Licença Sisbio: 27130
Data do registro                                                            01/11/2015
Data do evento                                                 02/11/2015 a 23/11/2015
Data de Carencia                                                            06/01/2016
Nome cientifico                            

### Taxonomic info:

Genero     36
Especie     7
Name: Nível Taxonômico, dtype: int64


### Taxonomy sample:

                                0                1                2
Filo                   Arthropoda       Arthropoda       Arthropoda
Classe                    Insecta          Insecta          Insecta
Ordem                 Lepidoptera      Lepidoptera      Lepidoptera
Familia                 Noctuidae        Noctuidae        Noctuidae
Genero                Helicoverpa      Helicoverpa      Helicoverpa
Especie           Sem Informações  Sem Informações  Sem Informações
Nível Taxonômico           Genero           Genero           Genero


In [27]:
####################################################################################
#
# Show sample of each output - filtered data

special_print("Filtered data info", biodiversity.filtered_info)
special_print("Filtered data sample", biodiversity.df_filtered.head(1).T)

   Nome da instituicao Sigla da instituicao Nome da base de dados  \
0                  NaN                  NaN                   NaN   
1                  NaN                  NaN                   NaN   
2                  NaN                  NaN                   NaN   
3                  NaN                  NaN                   NaN   
4                  NaN                  NaN                   NaN   
5                  NaN                  NaN                   NaN   
6                  NaN                  NaN                   NaN   
7                  NaN                  NaN                   NaN   
8                  NaN                  NaN                   NaN   
9                  NaN                  NaN                   NaN   
10                 NaN                  NaN                   NaN   
11                 NaN                  NaN                   NaN   
12                 NaN                  NaN                   NaN   
13                 NaN            

### Filtered data info:

File shape: 0 rows x 35 columns


### Filtered data sample:

                                           0
Nome da instituicao                      NaN
Sigla da instituicao                     NaN
Nome da base de dados                    NaN
Sigla da base de dados                   NaN
Responsavel pelo registro                NaN
Numero do registro no portal             NaN
Numero do registro na base de dados      NaN
Data do registro                         NaN
Data do evento                           NaN
Data de Carencia                         NaN
Nome cientifico                          NaN
Nome comum                               NaN
Nome cientifico na base de dados         NaN
Nivel taxonomico                         NaN
Numero de individuos                     NaN
Reino                                    NaN
Filo                                     NaN
Classe                                   NaN
Ordem                                    NaN
Familia                                  NaN
Genero                                   NaN
Especie   

In [20]:
####################################################################################
#
# Show sample of each output - show location info

special_print("Applied filters", FILTER_FIELDS)
special_print("Applied filter values", FILTER_VALUES)

special_print("Sample of locations to check", biodiversity.df_location_sample.head(1).T)

### Applied filters:

['Municipio', 'Filo']


### Applied filter values:

[['Nova Friburgo', 'Niquelândia', 'Vitoria', 'Primavera do Leste'], ['Mollusca', 'Magnoliophyta', 'Arthropoda']]


### Sample of locations to check:

                                                                               18
Nome da instituicao                                                           NaN
Sigla da instituicao                                                          NaN
Nome da base de dados                                                         NaN
Sigla da base de dados                                                        NaN
Responsavel pelo registro                                                     NaN
Numero do registro no portal                                                  NaN
Numero do registro na base de dados                                           NaN
Data do registro                                                              NaN
Data do evento                                                                NaN
Data de Carencia                                                              NaN
Nome cientifico                                                               NaN
Nome comum      

In [21]:
####################################################################################
#
# Show sample of each output - show map with reported observations

display(Markdown("### Observations (click to see more details)"))

### Observations (click to see more details)

In [22]:
biodiversity.observations_map